# DBSCAN demo
    -Density based spatial clustering of applications with noise
    -Only two chunks of data at this moemnt for initial test 



# Prepare the features and data pipeline for general machine learning study:
    Load the merged data into python pandas
    Build up the data structurte for machine learning algorithms, clutering in this experimental
    Apply DB Scan, Set Level Tree, etc

In [32]:
import pandas as pd
import numpy as np
import os
import csv

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn

sns.set(style="darkgrid")

import sys
print (sys.path)

['', '/opt/mapr/spark/spark-2.1.0/python', '/opt/mapr/spark/spark-2.1.0/python/lib/py4j-0.9-src.zip', '/mapr/datalake/optum/optuminsight/health_at_scale/python/lib/python3.5/site-packages', '/opt/mapr/tools/python/anaconda3/lib/python35.zip', '/opt/mapr/tools/python/anaconda3/lib/python3.5', '/opt/mapr/tools/python/anaconda3/lib/python3.5/plat-linux', '/opt/mapr/tools/python/anaconda3/lib/python3.5/lib-dynload', '/opt/mapr/tools/python/anaconda3/lib/python3.5/site-packages', '/opt/mapr/tools/python/anaconda3/lib/python3.5/site-packages/Sphinx-1.4.6-py3.5.egg', '/opt/mapr/tools/python/anaconda3/lib/python3.5/site-packages/setuptools-27.2.0-py3.5.egg', '/opt/mapr/tools/python/anaconda3/lib/python3.5/site-packages/IPython/extensions', '/home/hatscale/.ipython']


In [2]:
data_dir = '/mapr/datalake/uhclake/perf/Sqoop_RX/ymalcow/101_clustering_proj01/'
path_csvs = 'prep01ML_all2_csv'

path_mrgs = os.path.join(data_dir, path_csvs)

In [3]:
# mem_attrs: member attributes
# mem_attrs: merged claim attributes with members
mem_attrs = ['member_individual_identifier','src_cdb_xref_id','lgcy_src_id','social_security_number_hashed' , \
             'member_last_name','member_first_name' ,'member_middle_name','member_middle_initial_text', \
             'member_name_suffix', 'mem_dob_hashed', 'memeber_zip_code_of_residence','member_sex','addr_line_1',\
             'addr_line_2','city_name','state','zipcode','country']
mrg_attrs = ['claim_identifier', 'claim_system_identifier', 'member_identifier', 'member_individual_identifier', \
             'member_division_identifier', 'admission_src_code', 'icd_version_number', 'procedure_code', \
             'diag_code_diag_type_present_on_admis_ind', 'provider_medicare_identifier', 'drg', 'first_service_date', \
             'last_service_date', 'charged_amount', 'paid_amount', 'network_type', 'revenue_code', 'admission_type_code', \
             'hospital_discharge_code', 'place_of_service_code', 'billing_npi_identifier', 'rendering_npi_identifier', \
             'attending_npi_identifier', 'referring_npi_identifier', 'provider_mpin_number', 'provider_tin_identifier', \
             'provider_name', 'provider_address_street_text', 'provider_city_name', 'provider_state_code', \
             'provider_zip_code', 'mth', 'member_last_name', 'member_first_name', 'member_middle_name', \
             'member_middle_initial_text', 'member_name_suffix', 'member_age', 'member_zip_code_of_residence', \
             'member_income', 'member_income_type', 'member_alive_ind', 'tops_relationship_code', 'tops_sequence_number', \
             'member_marital_status', 'member_sex', 'member_race', 'member_coverage_effective_date', 
             'member_coverage_end_date', 'member_cosmos_panel_number', 'government_program_type_description', \
             'employee_status_type_description']

In [4]:
len(mrg_attrs)

52

In [4]:
# Define the list of attributes slected for machine learning study
ML_F_set1 = ['network_type', 'place_of_service_code', 'mth', 'member_age', \
             'member_sex', 'government_program_type_description', 'employee_status_type_description', 
             'charged_amount', 'paid_amount', 'member_income']
ML_F_set2 = ['network_type', 'mth', 'member_sex', 'government_program_type_description',
             'employee_status_type_description', 'charged_amount', 'paid_amount', 'member_income']

mrgs = os.listdir(path_mrgs)
mrgs

['.000029_0.crc',
 '000008_0',
 '000018_0',
 '000002_0',
 '.000013_0.crc',
 '.000005_0.crc',
 '000011_0',
 '000001_0',
 '.000019_0.crc',
 '.000043_0.crc',
 '.000042_0.crc',
 '.000048_0.crc',
 '.000000_0.crc',
 '.000026_0.crc',
 '.000028_0.crc',
 '000048_0',
 '.000014_0.crc',
 '.000015_0.crc',
 '000020_0',
 '000022_0',
 '000038_0',
 '000030_0',
 '.000017_0.crc',
 '000017_0',
 '000028_0',
 '000032_0',
 '.000007_0.crc',
 '.000022_0.crc',
 '000036_0',
 '.000044_0.crc',
 '.000020_0.crc',
 '.000006_0.crc',
 '000044_0',
 '000027_0',
 '000004_0',
 '000013_0',
 '000006_0',
 '000039_0',
 '000015_0',
 '000005_0',
 '000016_0',
 '000019_0',
 '.000024_0.crc',
 '.000025_0.crc',
 '.000009_0.crc',
 '000034_0',
 '000047_0',
 '000000_0',
 '000026_0',
 '000035_0',
 '.000012_0.crc',
 '000043_0',
 '.000038_0.crc',
 '000024_0',
 '.000002_0.crc',
 '.000030_0.crc',
 '000012_0',
 '.000036_0.crc',
 '.000047_0.crc',
 '.000034_0.crc',
 '.000039_0.crc',
 '.000046_0.crc',
 '.000041_0.crc',
 '000023_0',
 '.000001_0.c

In [9]:
# Read in the data chunk by chunk, select the slected features and concat the dataframe
df_all = pd.DataFrame(columns = ML_F_set1)
cnt = 0
for chunk in mrgs:
    if chunk[-3:] == 'crc':
        continue
    cnt += 1
    if cnt > 1:
        break
    curr = pd.read_csv(os.path.join(path_mrgs, chunk), sep = ';', names = mrg_attrs, error_bad_lines=False)
    curr_slt = curr[ML_F_set1]
    print ("Chunk {} contains {} records".format(cnt, len(curr_slt.index)))
    df_all = pd.concat([df_all, curr_slt])    

/opt/mapr/tools/python/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (13,16,19,24,30,37,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Chunk 1 contains 396521 records


In [10]:
print (len(df_all.index), '\n', df_all.dtypes)
df_all.head()

396521 
 network_type                            object
place_of_service_code                   object
mth                                     object
member_age                              object
member_sex                              object
government_program_type_description     object
employee_status_type_description        object
charged_amount                          object
paid_amount                            float64
member_income                          float64
dtype: object


,network_type,place_of_service_code,mth,member_age,member_sex,government_program_type_description,employee_status_type_description,charged_amount,paid_amount,member_income
0,IN,11,2018-02,76,FEMALE,MEDICARE,ACTIVE,0.01,0.0,0.0
1,IN,11,2018-02,76,FEMALE,MEDICARE,ACTIVE,0.05,0.0,0.0
2,IN,11,2018-02,76,FEMALE,MEDICARE,ACTIVE,0.02,0.0,0.0
3,IN,11,2018-02,76,FEMALE,MEDICARE,ACTIVE,130,0.0,0.0
4,IN,11,2018-01,57,MALE,MEDICARE,ACTIVE,205,0.0,0.0


save the merged dataframe with selected effective attributes to csv file
<br>
out_ml = 'ml2018_set1.csv'
path_out_ml = os.path.join(data_dir, out_ml)

df_all.to_csv(path_out_ml, sep `= ';')

os.listdir(data_dir)

In [11]:
df_all.mth.unique()

array(['2018-02', '2018-01', '2018-03', '2018-04', '019151705'],
      dtype=object)

In [12]:
df_all.count()

network_type                           396521
place_of_service_code                  396521
mth                                    396521
member_age                             396516
member_sex                             396521
government_program_type_description    396521
employee_status_type_description       396521
charged_amount                         396521
paid_amount                            396521
member_income                          396521
dtype: int64

In [13]:
df_all[df_all.charged_amount == '2018-01-11']

,network_type,place_of_service_code,mth,member_age,member_sex,government_program_type_description,employee_status_type_description,charged_amount,paid_amount,member_income


In [14]:
df_all.member_sex.unique()

array(['FEMALE', 'MALE', '\\N'], dtype=object)

In [15]:
df_all = df_all.reindex(columns = ML_F_set1)
df_all.columns

Index(['network_type', 'place_of_service_code', 'mth', 'member_age',
       'member_sex', 'government_program_type_description',
       'employee_status_type_description', 'charged_amount', 'paid_amount',
       'member_income'],
      dtype='object')

In [16]:
df_all.head()

,network_type,place_of_service_code,mth,member_age,member_sex,government_program_type_description,employee_status_type_description,charged_amount,paid_amount,member_income
0,IN,11,2018-02,76,FEMALE,MEDICARE,ACTIVE,0.01,0.0,0.0
1,IN,11,2018-02,76,FEMALE,MEDICARE,ACTIVE,0.05,0.0,0.0
2,IN,11,2018-02,76,FEMALE,MEDICARE,ACTIVE,0.02,0.0,0.0
3,IN,11,2018-02,76,FEMALE,MEDICARE,ACTIVE,130,0.0,0.0
4,IN,11,2018-01,57,MALE,MEDICARE,ACTIVE,205,0.0,0.0


In [17]:
%%time
# clean data

df_all['charged_amount'] = df_all['charged_amount'].astype(str)
df_all['charged_amount'] = df_all.charged_amount.apply(lambda x : str(x) if pd.notnull(x) else '')

CPU times: user 600 ms, sys: 5 ms, total: 605 ms
Wall time: 602 ms


In [18]:
df_all.dtypes

network_type                            object
place_of_service_code                   object
mth                                     object
member_age                              object
member_sex                              object
government_program_type_description     object
employee_status_type_description        object
charged_amount                          object
paid_amount                            float64
member_income                          float64
dtype: object

In [19]:
df_all = df_all[~df_all.charged_amount.str.contains('2018')]

In [20]:
len(df_all.index)

396511

In [17]:
# convert the data type
df_all.charged_amount.astype(float)

0            0.01
1            0.05
2            0.02
3          130.00
4          205.00
5           20.00
6           60.00
7         5121.00
8          360.00
9          360.00
10         190.00
11         257.00
12         129.00
13         219.00
14         350.00
15         170.00
16         165.00
17          90.00
18        1635.90
19         235.00
20         293.04
21         660.00
22         235.20
23         192.00
24         843.29
25         310.00
26         859.00
27         400.00
28         260.00
29         337.00
           ...   
388220     170.00
388221     175.00
388222      97.00
388223     235.00
388224     351.00
388225      19.00
388226     275.00
388227     118.57
388228     184.00
388229      52.00
388230     190.00
388231     180.00
388232     153.95
388233     117.00
388234     138.00
388235      66.00
388236      95.00
388237     328.33
388238     551.00
388239     484.00
388240     204.00
388241     235.00
388242     333.00
388243     181.00
388244    

# One-hot encoder

In [21]:
#Import OneHotEncoder (and LabelEncoder as helper)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [22]:
# Category features
CAT_F_set1 = ['network_type', 'place_of_service_code', 'mth', 'member_age', \
             'member_sex', 'government_program_type_description', 'employee_status_type_description']
CAT_F_set2 = ['network_type', 'mth', 'member_sex', 'government_program_type_description', \
               'employee_status_type_description']

In [23]:
# Clean cat features
for i in range(len(CAT_F_set1)):
    df_all[CAT_F_set1[i]] = df_all[CAT_F_set1[i]].apply(lambda x: str(x) if pd.notnull(x) else "")

In [24]:
# For machine learning modeling/clustering we are going to work on numericl data;
# So we need to ran one-hot encoding to convert attributes to binary vectors ([0,...,0,1,0,...0], \
# the 1 is corresponding to the value in the category)
# LabelEncoder ---> OneHotEncoder

labels = []

for i in range(len(CAT_F_set1)):
    UF = df_all[CAT_F_set1[i]].unique()
    print ("the current unique values for {} are:\n {}".format(CAT_F_set1[i], UF))
    labels.append(list(UF))

the current unique values for network_type are:
 ['IN' 'ADJIN' 'OUT|ADJIN' 'OUT' 'IN|ADJIN' 'OUT|IN']
the current unique values for place_of_service_code are:
 ['11' '21' '81' '23' '12' '22' '49' '65' '41' '31' '19' '24' '32' '2' '20'
 '53' '15' '61' '72' '60' '51' '52' '50' '54' '13' '17' '33' '71' '42'
 '14' '99' '3' '4' '62' '16' '18' '02' '01' '06' '56' '57' '34' '55' '26']
the current unique values for mth are:
 ['2018-02' '2018-01' '2018-03' '2018-04']
the current unique values for member_age are:
 ['76' '57' '71' '81' '74' '43' '62' '75' '73' '69' '77' '67' '63' '72'
 '29' '58' '82' '65' '70' '87' '66' '68' '90' '40' '41' '56' '28' '54'
 '86' '78' '52' '48' '59' '79' '83' '55' '84' '46' '61' '49' '64' '88'
 '85' '42' '53' '51' '94' '60' '80' '92' '39' '47' '50' '34' '22' '30'
 '44' '35' '95' '93' '96' '89' '45' '97' '36' '25' '24' '91' '37' '26'
 '33' '38' '31' '32' '98' '99' '18' '23' '21' '19' '27' '102' '100' '107'
 '16' '2' '15' '1' '3' '14' '8' '0' '5' '17' '20' '9' '7' '11

In [25]:
#One hot encode all categorical attributes
cats = []
for i in range(len(CAT_F_set1)):
    #Label encode
    print (i)
    label_encoder = LabelEncoder()
    label_encoder.fit(labels[i])
    feature = label_encoder.transform(df_all.iloc[:,i])
    feature = feature.reshape(df_all.shape[0], 1)
    #One hot encode
    onehot_encoder = OneHotEncoder(sparse=False,n_values=len(labels[i]))
    feature = onehot_encoder.fit_transform(feature)
    cats.append(feature)

0
1
2
3
4
5
6


In [35]:
# Make a 2D array from a list of 1D arrays
encoded_cats = np.column_stack(cats)

In [33]:
len(cats)

7

In [31]:
len(cats[3][1])

215

In [40]:
print (encoded_cats.shape)
encoded_cats[35]

(396511, 278)


array([0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [41]:
#Concatenate encoded attributes with continuous attributes
split = 7
dataset_encoded = np.concatenate((encoded_cats,df_all.iloc[:,split:].values),axis=1)

del encoded_cats
print(dataset_encoded.shape)

(396511, 281)


In [44]:
dataset_encoded[333]

array([0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [ ]:
del cats
del feature
del dataset

# Remove the high-dimensional category attributes to avoid memory issue
df_all2 = df_all[ML_F_set2]
df_all2.head()

df_all2 = df_all2.iloc[0:1000000]
df_all2.head()

# For machine learning modeling/clustering we are going to work on numericl data;
# So we need to ran one-hot encoding to convert attributes to binary vectors ([0,...,0,1,0,...0], \
# the 1 is corresponding to the value in the category)
# LabelEncoder ---> OneHotEncoder

labels2 = []

for i in range(len(CAT_F_set2)):
    UF2 = df_all2[CAT_F_set2[i]].unique()
    print ("the current unique values for {} are:\n {}".format(CAT_F_set2[i], UF2))
    labels2.append(list(UF2))

#One hot encode all categorical attributes
cats2 = []
for i in range(len(CAT_F_set2)):
    #Label encode
    print (i)
    label_encoder2 = LabelEncoder()
    label_encoder2.fit(labels2[i])
    feature2 = label_encoder2.transform(df_all2.iloc[:,i])
    feature2 = feature2.reshape(df_all2.shape[0], 1)
    #One hot encode
    onehot_encoder2 = OneHotEncoder(sparse=False,n_values=len(labels2[i]))
    feature2 = onehot_encoder.fit_transform(feature2)
    cats2.append(feature2)

# Check memory


In [39]:
# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

In [42]:
# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and \
        x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('df_all', 11688851630),
 ('df_all2', 9155443252),
 ('curr', 1003005024),
 ('curr_slt', 161304782),
 ('OneHotEncoder', 1304),
 ('SparkContext', 1304),
 ('StorageLevel', 1304),
 ('LabelEncoder', 1016),
 ('SQLContext', 1016),
 ('SparkSession', 1016),
 ('mrgs', 912),
 ('mrg_attrs', 480),
 ('mem_attrs', 208),
 ('ML_F_set1', 144),
 ('UF', 136),
 ('UF2', 136),
 ('path_mrgs', 133),
 ('ML_F_set2', 128),
 ('labels', 128),
 ('labels2', 128),
 ('CAT_F_set1', 120),
 ('data_dir', 116),
 ('feature', 112),
 ('feature2', 112),
 ('CAT_F_set2', 104),
 ('cats', 96),
 ('np', 80),
 ('pd', 80),
 ('plt', 80),
 ('sns', 80),
 ('path_csvs', 66),
 ('cats2', 64),
 ('chunk', 62),
 ('label_encoder', 56),
 ('label_encoder2', 56),
 ('onehot_encoder', 56),
 ('onehot_encoder2', 56),
 ('cnt', 28),
 ('i', 24)]

# Release unreferred memory
import gc
gc.collect()

In [45]:
# Delete inactive variables
biglist = ['df_all', 'curr, curr_slt']
# del biglist
del df_all


In [47]:
del curr
del curr_slt

In [57]:
# Get a sorted list of the objects and their sizes after some mem cleaning
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and \
        x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('df_all2', 401808192),
 ('OneHotEncoder', 1304),
 ('SparkContext', 1304),
 ('StorageLevel', 1304),
 ('LabelEncoder', 1016),
 ('SQLContext', 1016),
 ('SparkSession', 1016),
 ('mrgs', 912),
 ('mrg_attrs', 480),
 ('mem_attrs', 208),
 ('ML_F_set1', 144),
 ('UF', 136),
 ('path_mrgs', 133),
 ('ML_F_set2', 128),
 ('UF2', 128),
 ('labels', 128),
 ('labels2', 128),
 ('CAT_F_set1', 120),
 ('data_dir', 116),
 ('feature', 112),
 ('feature2', 112),
 ('CAT_F_set2', 104),
 ('cats', 96),
 ('cats2', 96),
 ('biglist', 80),
 ('np', 80),
 ('pd', 80),
 ('plt', 80),
 ('sns', 80),
 ('path_csvs', 66),
 ('chunk', 62),
 ('label_encoder', 56),
 ('label_encoder2', 56),
 ('onehot_encoder', 56),
 ('onehot_encoder2', 56),
 ('cnt', 28),
 ('i', 28)]

# Data exploration
1. Numeric fetures - cross correlation table using seaborn

In [48]:
NUM_F = ['member_income', 'charged_amount', 'paid_amount']

In [49]:
g = sns.pairplot(df_all[NUM_F])

In [ ]:
plt.show()

# Pyspark MLLIB test

In [59]:
import pyspark.mllib

In [60]:
import pybrain